In [1]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import pathlib
import datetime as dt
import time
import os
import networkit as nk
import tabulate
from IPython.display import IFrame
%matplotlib inline

# turn response caching on and turn on logging to your terminal window
ox.config(log_console=True, use_cache=True)

ox.__version__

'1.0.0'

In [2]:
places = [{'county' : 'Merida',
           'state' : 'Yucatan',
           'country' : 'Mexico'},
          {'county' : 'Kanasin',
           'state' : 'Yucatan',
           'country' : 'Mexico'}]

def get_roads_osmnx(places, update=False, proj=False, crs=None):

    dirpath = pathlib.Path('./networks/')
    filepath = dirpath/'merida-kanasin-road.graphml'
    logpath = dirpath/'log'
                                    
    if filepath.exists() and not update:
        G = ox.load_graphml(filepath)
    else:
        # get drivable public streets network, aka road network, without service roads,
        # e.g. private, parking lots, etc.
        # use retain_all if you want to keep all disconnected subgraphs (e.g. when your places aren't adjacent)
        # TODO: It would be nice to setup up a polygon for the city and its surrounding areas, to be sure
        # exactly the location.
        G = ox.graph_from_place(places, network_type='drive')
        ox.save_graphml(G, filepath=filepath, gephi=False)
        
    if proj:
        G = ox.project_graph(G, to_crs=crs)
    
    print(f"Graph created at: {G.graph['created_date']}")
    return G, *ox.graph_to_gdfs(G)
        
#G, nodes, edges = get_roads_osmnx(places, update=False)
G_proj, nodes_proj, edges_proj = get_roads_osmnx(places, update=False, proj=True, crs=3857)

Graph created at: 2021-02-17 11:57:58


In [3]:
print(f"The road network has {G_proj.number_of_edges()} edges and {G_proj.number_of_nodes()} nodes")

The road network has 102400 edges and 37906 nodes


In [4]:
## what sized area does our network cover in square meters?
graph_area_m = nodes_proj.unary_union.convex_hull.area
graph_area_km = graph_area_m / 1e6 # Squared meters? What is the oficial value?
graph_area_km # squared kilometers

1154.1817817111275

In [5]:
# get largest strongly connected component, for those metrics that require
# strongly connected graphs
Gs = ox.utils_graph.get_largest_component(G_proj, strongly=True)

In [6]:
print(f"The largest component has {Gs.number_of_edges()} edges and {Gs.number_of_nodes()} nodes")

The largest component has 102131 edges and 37746 nodes


In [7]:
# Convert MultiDiGraph to DiGraph.
# Chooses between parallel edges by minimizing weight attribute value.
D = ox.utils_graph.get_digraph(Gs, weight="length")

# create undirected Graph from the DiGraph, for those metrics that need it
Gu = nx.Graph(D)

In [13]:
# get the nodes and edges as geodataframes
nodes, edges = ox.graph_to_gdfs(Gs)

In [14]:
# store in a list the values of the street length that will be the weights of the graph
length_values = edges['length'].values

Dijkstra Single Source Shortest Path (NetworkX)

In [ ]:
start_time = time.time()

# precompute shortest paths between all nodes for eccentricity-based
# stats
length_func = nx.single_source_dijkstra_path_length
sp = {source: dict(length_func(Gs, source, weight="length")) for source in Gs.nodes}

utils.log("Calculated shortest path lengths")

end_time = time.time()-start_time
end_time

In [ ]:
#Another way to see outputs
length_func = nx.single_source_dijkstra_path_length
for source in Gs.nodes:
    path_nx = length_func(Gs, source, weight="length")
    print(source)
    print(path_nx)
    print(dict(path_nx))

# Using networkit

Convert from a NetworkX graph to a NetworKit graph

As NetworKit does not allow setting edges weights when converting a NetworkX multidigraph to a NetworKit graph, we need to convert it to an unweighted and later we can set weights.

In [9]:
# multidigraph from the largest strongest component
Gkit = nk.nxadapter.nx2nk(Gs, weightAttr=None)

In [10]:
# directed nk graph
Dkit = nk.nxadapter.nx2nk(D, weightAttr='length')

In [11]:
# undirected nk graph
Gukit = nk.nxadapter.nx2nk(Gu, weightAttr='length')

Now we can set the weights to the unweighted graph.

First, we need to transform it to a weighted graph and then assign it weights:

In [15]:
Gkit = nk.graphtools.toWeighted(Gkit)

# iterate through edges and set weights from a list
for i, (u, v) in enumerate(Gkit.iterEdges()):
		w = length_values[i] #list of weights
		Gkit.setWeight(u, v, w)

Now we can see that it is a weighted graph and observe some information of the different graphs:

In [16]:
nk.overview(Gkit)

Network Properties:
nodes, edges			37746, 102131
directed?			True
weighted?			True
isolated nodes			0
self-loops			83
density				0.000072
min/max/avg degree		1, 5, 2.705744
degree assortativity		0.382673
number of connected components	1
size of largest component	37746 (100.00 %)


In [17]:
nk.overview(Dkit)

Network Properties:
nodes, edges			37746, 101599
directed?			True
weighted?			True
isolated nodes			0
self-loops			44
density				0.000071
min/max/avg degree		1, 5, 2.691649
degree assortativity		0.384650
number of connected components	1
size of largest component	37746 (100.00 %)


In [18]:
nk.overview(Gukit)

Network Properties:
nodes, edges			37746, 58310
directed?			False
weighted?			True
isolated nodes			0
self-loops			44
density				0.000082
min/max/avg degree		1, 6, 3.088433
degree assortativity		0.263291
number of connected components	1
size of largest component	37746 (100.00 %)


### Dijkstra Single Source Shortest Path length (NetworKit)

In [19]:
Gkit_idx = Gkit

In [20]:
Gkit_idx.indexEdges()

In [ ]:
start_time = time.time()

#dijkstra = nk.distance.Dijkstra
sp = {}
for source in Gkit_idx.iterNodes():

    dijkstra = nk.distance.Dijkstra(Gkit_idx, source, True, True)
    dijkstra.run()

    path_length = dijkstra.getDistances()
    nodes_distance = dijkstra.getNodesSortedByDistance()
    
    print(source)
    print(path_length)
    print(nodes_distance)
    #sp[source] = dict(path_length)
		#{source: dict(path_length)}

end_time = time.time()-start_time
end_time

Eccentricity

The eccentricity of a node u is defined as the distance to the farthest node from node u. In other words, it is the longest shortest-path starting from node u.

The eccentricity of a graph can be computed by calling the `getValue(G, v)` method, and passing a graph and a node. The method returns the node farthest from v, and the length of the shortest path between v and the farthest node.

In [ ]:
start_time = time.time()

for source in Gkit_idx.iterNodes():
    nk.distance.Eccentricity.getValue(G, source)

end_time = time.time()-start_time
end_time

In [24]:
farthest_node, long_sp = nk.distance.Eccentricity.getValue(Gkit_idx, 0)

In [25]:
long_sp

159

In [26]:
farthest_node

33949

In [27]:
nk.distance.Eccentricity.getValue(Gkit_idx, 0)

(33949, 159)

Diameter

In [31]:
# Initialize algorithm to compute the exact diameter of the input graph
diam = nk.distance.Diameter(Gkit_idx,algo=1)

In [32]:
%%time
# Run
diam.run()

CPU times: user 8min 47s, sys: 233 ms, total: 8min 47s
Wall time: 8min 48s


In [33]:
diam.getDiameter()

(63535, 0)